1. Реализуйте класс AugmentationPipeline с методами:
   - add_augmentation(name, aug)
   - remove_augmentation(name)
   - apply(image)
   - get_augmentations()
2. Создайте несколько конфигураций (light, medium, heavy).
3. Примените каждую конфигурацию к train и сохраните результаты.

In [ ]:
import os
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
from datasets import CustomImageDataset
from extra_augs import AddGaussianNoise, CutOut

class AugmentationPipeline:
    def __init__(self):
        self.augmentations = {}
        self.order = []

    def add_augmentation(self, name, aug):
        self.augmentations[name] = aug
        self.order.append(name)

    def remove_augmentation(self, name):
        if name in self.augmentations:
            del self.augmentations[name]
            self.order.remove(name)

    def apply(self, image):
        # Применение аугментации
        for name in self.order:
            image = self.augmentations[name](image)
        return image

    def get_augmentations(self):
        return self.order

def build_pipeline(config='light'):
    pipeline = AugmentationPipeline()

    if config == 'light':
        pipeline.add_augmentation("Flip", transforms.RandomHorizontalFlip(p=0.5))
        pipeline.add_augmentation("ToTensor", transforms.ToTensor())

    elif config == 'medium':
        pipeline.add_augmentation("Flip", transforms.RandomHorizontalFlip(p=0.5))
        pipeline.add_augmentation("Crop", transforms.RandomCrop(200, padding=20))
        pipeline.add_augmentation("ColorJitter", transforms.ColorJitter(0.2, 0.2, 0.2, 0.1))
        pipeline.add_augmentation("ToTensor", transforms.ToTensor())
        pipeline.add_augmentation("Noise", AddGaussianNoise(0., 0.1))

    elif config == 'heavy':
        pipeline.add_augmentation("Flip", transforms.RandomHorizontalFlip(p=0.5))
        pipeline.add_augmentation("Crop", transforms.RandomCrop(200, padding=20))
        pipeline.add_augmentation("ColorJitter", transforms.ColorJitter(0.4, 0.4, 0.4, 0.2))
        pipeline.add_augmentation("ToTensor", transforms.ToTensor())
        pipeline.add_augmentation("Noise", AddGaussianNoise(0., 0.1))
        pipeline.add_augmentation("CutOut", CutOut(p=0.5, size=(32, 32)))

    return pipeline

def save_augmented_dataset(config_name, pipeline, data_root='data/train', save_root='augmented'):
    dataset = CustomImageDataset(data_root, transform=None, target_size=(224, 224))
    class_names = dataset.get_class_names()

    save_path_root = os.path.join(save_root, f"train_{config_name}")
    os.makedirs(save_path_root, exist_ok=True)
    for cls in class_names:
        os.makedirs(os.path.join(save_path_root, cls), exist_ok=True)

    print(f"Сохраняем изображения для конфигурации: {config_name}")

    for i in tqdm(range(len(dataset))):
        img, label = dataset[i]
        class_name = class_names[label]

        augmented_img = pipeline.apply(img)
        save_path = os.path.join(save_path_root, class_name, f"img_{i}.png")
        save_image(augmented_img, save_path)

# Выполнение всех конфигураций
for config_name in ['light', 'medium', 'heavy']:
    pipeline = build_pipeline(config_name)
    save_augmented_dataset(config_name, pipeline)

Сохраняем изображения для конфигурации: light


100%|██████████| 180/180 [00:02<00:00, 89.48it/s]


Сохраняем изображения для конфигурации: medium


100%|██████████| 180/180 [00:02<00:00, 77.53it/s]


Сохраняем изображения для конфигурации: heavy


100%|██████████| 180/180 [00:02<00:00, 78.87it/s]
